In [5]:
from datetime import datetime, timedelta

In [6]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

import pandas as pd

# 0. load data

### 1. transactions

In [7]:
'''
trans = get_dataframe_from_file("qgiv-stats-data", 'jess.stats.csv')
trans = trans[trans['year']==2023]
trans['date'] = pd.to_datetime(trans['date'])
trans['month'] = trans['date'].dt.month

print("Last run data cache:")
print("{:,} transactions; {:%Y-%m-%d} to {:%Y-%m-%d}".format(len(trans), trans['date'].min(), trans['date'].max()))
'''
print("NOT LOADING PRECAULCULATED DATA")

NOT LOADING PRECAULCULATED DATA


In [8]:
# onetime trans
q = '''select * 
        from transactions 
        where
            status='A' and
            recurring=0 and
            year=2024'''
trans_onetime = redshift_query_read(q, schema='production')
trans_onetime['date'] = pd.to_datetime(trans_onetime['date'])
trans_onetime['month'] = trans_onetime['date'].dt.month

print("{:,} transactions; {:%Y-%m-%d} to {:%Y-%m-%d}".format(len(trans_onetime), trans_onetime['date'].min(), trans_onetime['date'].max()))

1,114,114 transactions; 2024-01-01 to 2024-07-31


In [9]:
# recurring trans: count, mean, median
q = '''select
            t.*
        from transactions as t
        inner join (select
                        recurring,
                        min(date) as min_date
                    from transactions
                    group by recurring) as r on t.recurring=r.recurring and t.date=r.min_date
        where
            t.status='A' and
            t.recurring!=0 and
            r.min_date>='2024-01-01'
        order by date asc;'''
rec = redshift_query_read(q, schema='production')
rec = rec.groupby('recurring').first()

print("{:,} recurring; {:%Y-%m-%d} to {:%Y-%m-%d}".format(len(rec), rec['date'].min(), rec['date'].max()))

51,698 recurring; 2024-01-01 to 2024-07-31


In [10]:
trans = pd.concat([trans_onetime, rec])
print("{:,} transactions; {:%Y-%m-%d} to {:%Y-%m-%d}".format(len(trans), trans['date'].min(), trans['date'].max()))

1,165,812 transactions; 2024-01-01 to 2024-07-31


### 2. traffic

In [11]:
tables = ['ga', 'ga4_traffic', 'ga4_traffic_daily', 'ga4_traffic_daily_device',
          'ga4_traffic_weekly', 'ga4_traffic_weekly_device']
q = '''select
            form,
            date_trunc('month', date) as month,
            sum(views) as pageviews
        from {}
        where date>='2024-01-01'
        group by date_trunc('month', date), form'''
q_week = '''select
            form,
            date_trunc('month', week) as month,
            sum(views) as pageviews
        from {}
        where week>='2024-01-01'
        group by date_trunc('month', week), form'''

monthyear_data = None
for table in tables:
    if 'week' in table:
        _df = redshift_query_read(q_week.format(table), schema='production')
    else:
        _df = redshift_query_read(q.format(table), schema='production')
    
    _df['table'] = table
    monthyear_data = pd.concat([monthyear_data, _df])

In [12]:
monthyear_max = monthyear_data.groupby(['month', 'form'])['pageviews'].max().reset_index()
monthyear_max.sort_values('month', inplace=True)
monthyear_max.tail(2)

,month,form,pageviews
47648,2024-07-01,1003466,20
53490,2024-07-01,1039543,13


In [15]:
traff = monthyear_max.copy()
traff['date'] = traff['month']
traff.drop('month', axis=1, inplace=True)

In [17]:
print("{:,} entries; {:%Y-%m-%d} to {:%Y-%m-%d}".format(len(traff), traff['date'].min(), traff['date'].max()))

53,491 entries; 2024-01-01 to 2024-07-01


### 3. new forms extra settings

In [18]:
# new forms extra settings 
q = '''select 
            date as week, 
            form, 
            appearance, 
            conditional_fields, 
            image_size
        from analyticsqgiv_weekly
        where date_part('year', date)=2024
        order by date asc;'''
nuform_extra = redshift_query_read(q, schema="public")

In [19]:
print("{:,} entries; {} to {}".format(len(nuform_extra), nuform_extra['week'].min(), nuform_extra['week'].max()))

1,906,749 entries; 2024-01-01 00:00:00 to 2024-07-29 00:00:00


In [20]:
# set week start to trans date
trans['week'] = trans['date'].dt.to_period('w')
nuform_extra['week'] = nuform_extra['week'].dt.to_period('w')

In [21]:
trans = trans.merge(nuform_extra, on=['form', 'week'], how='left')
print("{:,} transactions".format(len(trans)))

1,165,812 transactions


In [22]:
# conditional fields: count forms, sum on/off, conversion on/off
trans['is_cond_fields'] = trans['conditional_fields']>0

In [23]:
# single/multi step: count forms, sum, conversion
# multistep: appearance==2
trans['is_multistep'] = trans['appearance']==2

In [24]:
# cleanup new forms extra settings columns
drop_cols = ['appearance', 'conditional_fields', 'image_size']
trans.drop(drop_cols, axis=1, inplace=True)

### 4. embeds

In [25]:
q = "select widget as form from embed"
embeds = redshift_query_read(q, schema='production')

trans['is_embed'] = trans['form'].isin(embeds['form'].tolist())

In [26]:
print("{:,} embed forms".format(len(embeds)))
print("{:,} embed transactions".format(len(trans[trans['is_embed']])))

21,643 embed forms
327,832 embed transactions


### 5. smart amounts

In [27]:
# log upgrades
q = '''select 
            form, 
            created, 
            message 
        from syslog_logs 
        where message like '%Smart Amount%' '''
logs = redshift_query_read(q, schema='production')

print("{:,} smart amounts log entries; {:,} forms".format(len(logs), len(logs['form'].unique())))
print("ranging from {:%Y-%m-%d} to {:%Y-%m-%d}".format(logs['created'].min(), logs['created'].max()))

1,398 smart amounts log entries; 963 forms
ranging from 2022-11-07 to 2024-07-31


In [28]:
logs['enabled'] = logs['message'].str.contains('Enabled').fillna(False)
logs['disabled'] = logs['message'].str.contains('Disabled').fillna(False)
entry_counts = logs.groupby('form')[['enabled', 'disabled']].sum().reset_index()

print("{:,} forms have only one entry (enabled)".format(len(entry_counts[(entry_counts['enabled']==1)&(entry_counts['disabled']==0)])))
print("{:,} forms enabled == disabled".format(len(entry_counts[entry_counts['enabled']==entry_counts['disabled']])))

433 forms have only one entry (enabled)
291 forms enabled == disabled


In [29]:
last_run_trans = get_dataframe_from_file('qgiv-stats-data', 'jess.stats.csv')
print("last run max date: {}".format(last_run_trans['date'].max()))

last run max date: 2024-02-05


In [30]:
# isolate smart amounts forms to more quickly tag transactions
new_trans = trans[trans['date']>=last_run_trans['date'].max()]

new_trans['is_smart_amounts'] = False
trans_sa_forms = new_trans[(new_trans['form'].isin(logs['form'].unique()))&(new_trans['date']>='2022-11-01')].copy()
trans_nonsa_forms = new_trans[(~new_trans['form'].isin(logs['form'].unique()))|(new_trans['date']<'2022-11-01')].copy()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
def is_using_smart_amounts(r):
    try:
        return logs[(logs['form']==r['form'])&(logs['created']<=r['date'])].sort_values('created', ascending=True)['enabled'].iloc[0]
    except:
        return False

In [33]:
# tag smart amounts transactions not already tagged in 
#   cached, tagged data
print("iterating through {:,} forms to tag smart amounts".format(len(trans_sa_forms['form'].unique())))
counter = 0
trans_sa_forms_tagged = None
for f in trans_sa_forms['form'].unique().tolist():
    these_trans = trans_sa_forms[trans_sa_forms['form']==f].copy()
    
    if f in entry_counts[(entry_counts['enabled']==1)&(entry_counts['disabled']==0)]['form'].tolist():
        # forms that were just enabled and never disabled
        activation_date = logs[logs['form']==f]['created'].iloc[0]
        these_trans['is_smart_amounts'] = these_trans['date']>=activation_date
    else:
        # forms that were enabled and disabled, possibly multiple times
        # need to verify transactions fall within active windows
        these_trans['is_smart_amounts'] = these_trans.apply(is_using_smart_amounts, axis=1)
    
    if trans_sa_forms_tagged is None:
        trans_sa_forms_tagged = these_trans
    else:
        trans_sa_forms_tagged = pd.concat([trans_sa_forms_tagged, these_trans])
    
    counter += 1
    if counter % 100 == 0:
        print("done w/ {} forms".format(counter))

iterating through 580 forms to tag smart amounts
done w/ 100 forms
done w/ 200 forms
done w/ 300 forms
done w/ 400 forms
done w/ 500 forms


In [34]:
print("{:,} tagged unique transaction ids".format(len(trans_sa_forms_tagged['id'].unique())))
print("{:,} tagged transactions".format(len(trans_sa_forms_tagged)))
print("{:,} transactions that used smart amounts".format(len(trans_sa_forms_tagged[trans_sa_forms_tagged['is_smart_amounts']]['id'].unique())))

print("{} last date for tagged transactions".format(trans_sa_forms_tagged[trans_sa_forms_tagged['is_smart_amounts']]['date'].max()))

40,028 tagged unique transaction ids
40,028 tagged transactions
31,666 transactions that used smart amounts
2024-07-31 00:00:00 last date for tagged transactions


In [35]:
trans = pd.concat([last_run_trans, trans_sa_forms_tagged, trans_nonsa_forms])
print("{:,} transactions".format(len(trans)))
print("{:,} smart amounts transactions".format(len(trans[trans['is_smart_amounts']])))
print("{:,} smart amounts transactions unique ids".format(len(trans[trans['is_smart_amounts']]['id'].unique())))

1,177,262 transactions
38,695 smart amounts transactions
38,383 smart amounts transactions unique ids


In [36]:
trans['date'] = pd.to_datetime(trans['date'])
trans.drop_duplicates(inplace=True)
print("{:,} transactions".format(len(trans)))
print("{:,} transactions unique ids".format(len(trans['id'].unique())))
print("{} max date".format(trans['date'].max()))

1,171,537 transactions
1,165,812 transactions unique ids
2024-07-31 00:00:00 max date


In [37]:
save_dataframe_to_file('qgiv-stats-data', 'jess.stats.2024.csv', trans)
#trans = get_dataframe_from_file("qgiv-stats-data", 'jess.stats.csv')

uploading to S3
Done


In [38]:
trans.groupby('month')['id'].count(), trans.groupby('month')['id'].nunique()

(month
 1    136575
 2    175443
 3    208498
 4    215376
 5    166548
 6    132953
 7    136144
 Name: id, dtype: int64,
 month
 1    136575
 2    169718
 3    208498
 4    215376
 5    166548
 6    132953
 7    136144
 Name: id, dtype: int64)

# 0. conversion

In [61]:
traff_grpd = traff.groupby(['form', 'date'])['pageviews'].sum().reset_index()
traff_grpd['month'] = traff_grpd['date'].dt.month
traff_grpd.tail(2)

,form,date,pageviews,month
53489,1039522,2024-07-01,20,7
53490,1039543,2024-07-01,13,7


In [55]:
form_month_totals = trans.groupby(['form', 'month'])['amount'].agg(['count', 'sum']).reset_index()
form_month_totals['volume'] = form_month_totals['sum']
form_month_totals.drop('sum', axis=1, inplace=True)
form_month_totals.tail(2)

,form,month,count,volume
47035,1039600,7,1,10.4
47036,1039626,7,1,0.0


In [63]:
form_month_mrgd = form_month_totals.merge(traff_grpd[['form', 'month', 'pageviews']], on=['form', 'month'])
form_month_mrgd['conversion'] = form_month_mrgd['count'] / form_month_mrgd['pageviews']
form_month_mrgd.tail(2)

,form,month,count,volume,pageviews,conversion
30410,1039451,7,2,12.0,40,0.050000
30411,1039497,7,1,500.0,7,0.142857


In [65]:
print("average conversion:")
form_month_mrgd['conversion'].agg(['mean', 'median'])

average conversion:


mean      0.390704
median    0.073171
Name: conversion, dtype: float64

In [68]:
forms_p2p = trans[trans['source']=='p2p']['form'].unique()
forms_qgiv = trans[trans['source']=='don_form']['form'].unique()

print("{:,} p2p forms; {:,} year round forms".format(len(forms_p2p), len(forms_qgiv)))

1,626 p2p forms; 12,579 year round forms


In [69]:
form_month_mrgd['is_p2p'] = form_month_mrgd['form'].isin(forms_p2p)
form_month_mrgd['is_qgiv'] = form_month_mrgd['form'].isin(forms_qgiv)

## universal conversion

In [73]:
print('p2p conversion')
print("aggregate:")
print(form_month_mrgd[form_month_mrgd['is_p2p']]['conversion'].agg(['mean', 'median']))

print()
print("cumulative:")
pageviews_p2p = form_month_mrgd[form_month_mrgd['is_p2p']]['pageviews'].sum()
trans_p2p = form_month_mrgd[form_month_mrgd['is_p2p']]['count'].sum()
print("{:.2f}%".format((trans_p2p / pageviews_p2p) * 100.))

p2p conversion
aggregate:
mean      0.064237
median    0.034682
Name: conversion, dtype: float64

cumulative:
3.16%


In [74]:
print('year round conversion')
print("aggregate:")
print(form_month_mrgd[form_month_mrgd['is_qgiv']]['conversion'].agg(['mean', 'median']))

print()
print("cumulative:")
pageviews_qgiv = form_month_mrgd[form_month_mrgd['is_qgiv']]['pageviews'].sum()
trans_qgiv = form_month_mrgd[form_month_mrgd['is_qgiv']]['count'].sum()
print("{:.2f}%".format((trans_qgiv / pageviews_qgiv) * 100.))

year round conversion
aggregate:
mean      0.422599
median    0.089404
Name: conversion, dtype: float64

cumulative:
5.50%


## representative forms

In [75]:
rep_forms = rep_forms = pd.read_csv("../representative forms/filtered_forms.csv")

In [81]:
_df = form_month_mrgd[(form_month_mrgd['is_p2p'])&(form_month_mrgd['conversion']<1.)]
_df = _df[_df['form'].isin(rep_forms['form'].tolist())]

print('p2p conversion')
print("aggregate:")
print(_df['conversion'].agg(['mean', 'median']))

print()
print("cumulative:")
pageviews_p2p = _df[_df['is_p2p']]['pageviews'].sum()
trans_p2p = _df[_df['is_p2p']]['count'].sum()
print("{:.2f}%".format((trans_p2p / pageviews_p2p) * 100.))

p2p conversion
aggregate:
mean      0.059144
median    0.037914
Name: conversion, dtype: float64

cumulative:
3.31%


In [80]:
_df = form_month_mrgd[(form_month_mrgd['is_qgiv'])&(form_month_mrgd['conversion']<1.)]
_df = _df[_df['form'].isin(rep_forms['form'].tolist())]

print('year round conversion')
print("aggregate:")
print(_df[_df['is_qgiv']]['conversion'].agg(['mean', 'median']))

print()
print("cumulative:")
pageviews_qgiv = _df[_df['is_qgiv']]['pageviews'].sum()
trans_qgiv = _df[_df['is_qgiv']]['count'].sum()
print("{:.2f}%".format((trans_qgiv / pageviews_qgiv) * 100.))

year round conversion
aggregate:
mean      0.139494
median    0.084211
Name: conversion, dtype: float64

cumulative:
6.05%


# 1. donation forms

- Total active donation forms
- Total amount raised
- \# of one-time donations
- Avg. one-time gift size
- \# of recurring donations
- Avg. recurring gift size

In [39]:
q = '''select
            date_trunc('month', date) as month,
            count(distinct(case when recurring=0 then id else null end)) as transactions_onetime,
            sum(amount) as volume,
            avg(case when recurring=0 then amount else null end) as mean_onetime,
            count(distinct(form)) as forms
        from transactions
        where
            status='A' and
            date>='2024-01-01' 
        group by date_trunc('month', date)'''
trans_onetime = redshift_query_read(q, schema='production')

q = '''select
            date_trunc('month', date) as month,
            median(amount) as median_onetime
        from transactions
        where
            status='A' and
            date>='2024-01-01' and
            recurring=0
        group by date_trunc('month', date)'''
mdn_onetime = redshift_query_read(q, schema='production')

trans_onetime = trans_onetime.merge(mdn_onetime, on='month')
#trans_onetime.sort_values('month').transpose()
trans_onetime.sort_values('month')

,month,transactions_onetime,volume,mean_onetime,forms,median_onetime
7,2024-01-01,128511,28429277.29,161.412180,9376,50.00
1,2024-02-01,164094,31638587.27,147.850985,10248,50.00
4,2024-03-01,197393,40868435.23,163.206656,10991,50.00
6,2024-04-01,207400,44134696.09,166.375902,11635,50.00
3,2024-05-01,160893,42285115.76,214.061767,11668,50.00
2,2024-06-01,126778,33045754.63,197.859049,11420,51.00
5,2024-07-01,129177,34572184.42,193.107132,11693,50.00
0,2024-08-01,6942,2430471.01,198.854190,3183,51.75


## onetime

In [40]:
'''
onetime
- Total active donation forms
- Total amount raised
- # of one-time donations
- Avg. one-time gift size
'''
aggs = ['count', 'sum', 'mean', 'median']

onetime = trans[trans['recurring']==0].groupby('month')['amount'].agg(aggs).reset_index()
onetime = onetime.merge(trans[trans['recurring']==0].groupby('month')['form'].nunique().reset_index(), on='month')
onetime.columns = ['month', 'transactions', 'volume', 'mean', 'median', 'forms']
#onetime.transpose()
onetime

,month,transactions,volume,mean,median,forms
0,1,128511,20743240.72,161.412180,50.00,5121
1,2,169577,25185501.99,148.519563,49.99,5896
2,3,197393,32215851.38,163.206656,50.00,6608
3,4,207400,34506362.04,166.375902,50.00,7180
4,5,160893,34441039.91,214.061767,50.00,7103
5,6,126778,25084174.55,197.859049,51.00,6693
6,7,129045,24923817.22,193.140511,50.00,6882


## recurring

In [41]:
'''
recurring
- # of recurring donations
- Avg. recurring gift size
'''
print("Recurring")
trans[trans['recurring']!=0].groupby('month')['amount'].agg(['count', 'mean', 'median']).reset_index()

Recurring


,month,count,mean,median
0,1,8064,84.705651,36.05
1,2,5866,72.339708,31.05
2,3,11105,58.040439,25.00
3,4,7976,61.536126,25.00
4,5,5655,117.744156,26.25
5,6,6175,100.520949,26.00
6,7,7099,353.573693,50.00


## gift assist

In [42]:
# gift assist
# perc of donors who use giftassist
ga = trans[trans['gift_assist_count']!=0].groupby('month')[['id', 'gift_assist_count']].agg({'gift_assist_count': 'sum', 'id': 'count'}).reset_index()
ga['perc'] = ga['gift_assist_count'] / ga['id']

print("Cumulative (all transactions):")
ga

Cumulative (all transactions):


,month,gift_assist_count,id,perc
0,1,42464,42464,1.0
1,2,49922,49922,1.0
2,3,52955,52955,1.0
3,4,55266,55266,1.0
4,5,47988,47988,1.0
5,6,39415,39415,1.0
6,7,40249,40249,1.0


In [43]:
# gift assist
# perc of donors who use giftassist
ga = trans[trans['gift_assist_count']!=0].groupby(['month', 'form'])[['id', 'gift_assist_count']].agg({'gift_assist_count': 'sum', 'id': 'count'}).reset_index()
ga['perc'] = ga['gift_assist_count'] / ga['id']

print("Aggregate (grouped by form):")
ga.groupby('month')['perc'].agg(['mean', 'median']).reset_index()

Aggregate (grouped by form):


,month,mean,median
0,1,1.0,1.0
1,2,1.0,1.0
2,3,1.0,1.0
3,4,1.0,1.0
4,5,1.0,1.0
5,6,1.0,1.0
6,7,1.0,1.0


## smart amounts

In [44]:
'''
smart amounts
- # of orgs using Smart Amounts
- Avg raised using Smart Amounts
'''
smartamounts = trans[trans['is_smart_amounts']].groupby('month')['org'].nunique().reset_index()

smartamounts_med = trans[trans['is_smart_amounts']].groupby(['month', 'org'])['amount'].sum().reset_index()
smartamounts.merge(smartamounts_med.groupby('month')['amount'].agg(['mean', 'median']).reset_index(), on='month')

,month,org,mean,median
0,1,89,8266.668090,1555.000
1,2,113,4950.537611,1065.180
2,3,144,6289.407431,1414.980
3,4,181,6805.965525,1603.150
4,5,207,4355.429179,1590.000
5,6,226,4334.837168,1562.765
6,7,258,3683.524457,1078.845


In [45]:
# not using smart amounts
notsmartamounts = trans[~trans['is_smart_amounts']].groupby(['month', 'org'])['amount'].sum().reset_index()
notsmartamounts.groupby('month')['amount'].agg(['mean', 'median']).reset_index()

,month,mean,median
0,1,8434.803763,1328.00
1,2,9350.666655,1500.00
2,3,10999.902165,1881.00
3,4,10466.613273,1747.70
4,5,10786.915569,1647.04
5,6,7912.069827,1300.00
6,7,8044.801810,1250.00


## conditional fields

In [46]:
'''
conditional fields
- # of forms using conditional fields
- $s raised by forms using conditional fields
- $s raised by forms not using conditional fields
- Completion rate of forms using conditional fields
- Completion rate of forms not using conditional fields
'''
cond_stats = trans[trans['source']=='don_form'].groupby(['month', 'is_cond_fields'])[['form', 'amount']].agg({'form': 'nunique', 'amount': 'sum'}).reset_index()

condfields_forms = trans[trans['source']=='don_form'].groupby(['date', 'form']).agg({'id': 'count', 'is_cond_fields': 'median'}).reset_index()
condfields_forms['date'] = pd.to_datetime(condfields_forms['date'])
condfields_forms = condfields_forms.merge(traff, on=['date', 'form'], how='left').fillna(0)
condfields_forms['month'] = condfields_forms['date'].dt.month
condfields_months = condfields_forms.groupby(['month', 'is_cond_fields'])[['id', 'views']].sum().reset_index()
condfields_months['conversion'] = condfields_months['id'] / condfields_months['views']
condfields_conv = condfields_months.pivot(index='month', columns='is_cond_fields', values='conversion')
condfields_conv = condfields_conv.reset_index()
condfields_conv_cols = ['month', 'no cond fields conv', 'cond fields conv']
for c in condfields_conv.columns:
    if str(c) not in ['0.0', '1.0'] and c != 'month':
        condfields_conv.drop(c, axis=1, inplace=True)
condfields_conv.columns = condfields_conv_cols

cond_stats = cond_stats.merge(condfields_conv, on='month')

cond_stats = cond_stats.pivot(index='month', columns='is_cond_fields', values=['form', 'amount', 'no cond fields conv', 'cond fields conv']).reset_index()

cond_stats.columns = ['month', 'forms not using condfields', 
                      'forms using condfields', 'volume not condfields',
                      'volume condfields', 'not condfields conv',
                      'not condfields conv 2', 'condfields conv',
                      'condfields conv 2']
cond_stats[['month', 'forms not using condfields', 
            'forms using condfields', 'volume not condfields',
            'volume condfields', 'not condfields conv', 
            'condfields conv']]

KeyError: "Columns not found: 'views'"

## single step vs multi step

In [ ]:
'''
single step vs multi step
- # of single step forms
- $s raised by single step forms
- Completion rate of single step forms
- # of multi-step forms
- $s raised by multi-step forms
- Completion rate of multi-step forms
'''
steps = trans.groupby(['month', 'is_multistep'])[['form', 'amount']].agg({'form': 'nunique', 'amount': 'sum'}).reset_index()
steps = steps.pivot(index='month', columns='is_multistep', values=['form', 'amount']).reset_index()
steps.fillna(0, inplace=True)
steps.columns = ['month', 'singlestep forms', 'multistep forms', 'singlestep volume', 'multistep volume']

multistep_forms = trans[trans['source']=='don_form'].groupby(['date', 'form']).agg({'id': 'count', 'is_multistep': 'first'}).reset_index()
multistep_forms['date'] = pd.to_datetime(multistep_forms['date'])
multistep_forms = multistep_forms.merge(traff, on=['date', 'form'], how='left').fillna(0)
multistep_forms['month'] = multistep_forms['date'].dt.month
multistep_month = multistep_forms.groupby(['month', 'is_multistep'])[['id', 'views']].sum().reset_index()
multistep_month['conversion'] = multistep_month['id'] / multistep_month['views']
multistep_conv = multistep_month.pivot(index='month', columns='is_multistep', values='conversion')
multistep_conv = multistep_conv.reset_index()
multistep_conv.columns = ['month', 'single step conv', 'multi step conv']

steps.merge(multistep_conv, on='month')

## embed

In [ ]:
embed_agg = embed_forms.groupby(['month', 'is_embed'])['sum'].agg(['sum', 'mean', 'median']).reset_index()
embed_agg = embed_agg.merge(embed_forms[embed_forms['count']<=embed_forms['views']].groupby(['month', 'is_embed'])['conversion'].mean(), on=['month', 'is_embed'])
embed_agg = embed_agg.pivot(index='month', columns='is_embed', values=['sum', 'mean', 'median', 'conversion'])
embed_agg.head()

In [ ]:
'''
embeds
- $$ raised by embedded forms
- $$ raised by non-embedded forms
- Avg raised using embedded forms
- Median raised using embedded forms
- Avg raised using non-embedded forms
- Median raised using non-embedded forms
- Completion rate of embedded forms
- Completion rate of non-embedded forms
'''
monthly_forms = trans.groupby(['month', 'form'])['amount'].agg(['count', 'sum']).reset_index()
embed_forms = trans.groupby(['month', 'form'])['is_embed'].mean().reset_index()
embed_forms = monthly_forms.merge(embed_forms, on=['month', 'form'])

traff['month'] = traff['date'].dt.month
embed_forms = embed_forms.merge(traff.groupby(['month', 'form'])['views'].sum().reset_index(), on=['month', 'form'])
embed_forms['conversion'] = embed_forms['count'] / embed_forms['views']
embed_forms['is_embed'] = embed_forms['is_embed'].round(0)

embed_agg = embed_forms.groupby(['month', 'is_embed'])['sum'].agg(['sum', 'mean', 'median']).reset_index()
embed_agg = embed_agg.merge(embed_forms[embed_forms['count']<=embed_forms['views']].groupby(['month', 'is_embed'])['conversion'].mean(), on=['month', 'is_embed'])
embed_agg = embed_agg.pivot(index='month', columns='is_embed', values=['sum', 'mean', 'median', 'conversion']).reset_index()

embed_agg.columns = ['month', 'nonembed volume', 'embed volume',
                     'non embed mean', 'embed mean', 'non embed median',
                     'embed median', 'non embed conversion', 'embed conversion']
embed_agg

# 2. standard events

- Total \# of standard events
- Total amount raised (reg. + donations)
- Total registrations
- Total donations (online & offline)

In [ ]:
q = '''select
            date_trunc('month', date) as month,
            count(distinct(id)) as registrations_count,
            sum(donations_count) as donations_count,
            sum(amount) as total_events,
            sum(amount)-sum(donations_amt) as registrations_volume,
            sum(donations_amt) as donations_volume
        from transactions
        where
            status='A' and
            events_amt>0 and
            date>='2024-01-01'
        group by date_trunc('month', date)'''
events = redshift_query_read(q, schema='production')
events.sort_values('month')

In [ ]:
events_months = trans[trans['events_count']>0].groupby('month')[['events_amt', 'donations_amt']].agg(['sum', 'count']).reset_index()
events_months.columns = ['month', 'registrations volume', 'registrations counts',
                       'donations volume', 'donations counts']

# get events from qgiv analytics
q = '''select date, sum(events_count)
        from analyticsqgiv_weekly
        where date>=2024
        group by date;'''
events = redshift_query_read(q, schema='public')
events['month'] = events['date'].dt.month
event_counts_months = events.groupby('month')['sum'].agg(['mean', 'median']).reset_index()
event_counts_months.columns = ['month', 'mean events count', 'median events count']

events_months.merge(event_counts_months, how='outer')

# 3. text fundraising

- Total \# of orgs using text fundraising
- Total amount raised
- Avg. gift size through text fundraising

In [ ]:
q = '''select
            date_trunc('month', date) as month,
            count(distinct(id)) as transactions,
            sum(amount) as volume,
            avg(amount) as mean_amount,
            count(distinct(org)) as orgs
        from transactions
        where status='A' and date>='2024-01-01' and source='sms'
        group by date_trunc('month', date);'''
sms = redshift_query_read(q, schema='production')

q = '''select
            date_trunc('month', date) as month,
            median(amount) as median_amount
        from transactions
        where status='A' and date>='2024-01-01' and source='sms'
        group by date_trunc('month', date);'''
sms_median = redshift_query_read(q, schema='production')

sms = sms.merge(sms_median, on='month')

In [ ]:
sms.sort_values('month')

In [ ]:
q = '''select 
            date_part('month', created) as month, 
            count(distinct(id)) as campaigns
        from smscampaign
        where created>=2024
        group by date_part('month', created)'''
campaigns_monthly = redshift_query_read(q, schema='production')

In [ ]:
campaigns_monthly

# 4. P2P

- Total \# of peer-to-peer events
- Total amount raised (reg. + donations)
- Total registrations
- Total donations (online & offline)

In [ ]:
q = '''select
        t.month,
        count(distinct(t.form)) as active_p2p_events,
        sum(t.amount) as p2p_vol,
        count(t.id) as p2p_count,
        sum(t.registrations_count) as p2p_registrations,
        sum(t.registrations_amt) as p2p_registrations_vol,
        sum(t.donations_amt) as p2p_donations_vol,
        sum(t.donations_count) as p2p_donations_count
    from transactions as t
        left join form as f on t.form=f.id
    where
        t.status='A' and
        t.recurring=0 and
        f.type=3 and
        t.year>=2024
    group by t.month'''
df_p2p = redshift_query_read(q, schema='production')

In [ ]:
df_p2p.sort_values('month', ascending=True, inplace=True)

In [ ]:
# teams
# total teams count
q = '''select
            date,
            sum(teams_count) as teams_count
        from analyticsp2p_weekly
        where date>='2024-01-01'
        group by date'''
teams_date = redshift_query_read(q, schema='public')
teams_date['month'] = teams_date['date'].dt.month
teams_monthly = teams_date.groupby('month')['teams_count'].agg(['mean', 'median']).reset_index()
teams_monthly.columns = ['month', 'mean teams', 'median teams']

# total raised from teams
teams_vol = trans[trans['transdonationentitytype']==2].groupby('month')['amount'].sum().reset_index()
teams_vol.columns = ['month', 'volume raised']

teams_monthly = teams_monthly.merge(teams_vol, on=['month'])

In [ ]:
teams_vol

In [ ]:
df_p2p = df_p2p.merge(teams_monthly, on='month')

In [ ]:
# badges
q = '''select 
            date_part('month', created) as month,
            org
        from logs 
        where 
            message like '%badge%' and
            created>=2024
        group by date_part('month', created), org'''
badge_orgs = redshift_query_read(q, schema='production')

In [ ]:
monthly_vol = []
for month in badge_orgs['month'].unique():
    these_orgs = badge_orgs[badge_orgs['month']==month]['org'].tolist()
    
    monthly_vol.append({
        'month': month,
        'orgs badges': len(these_orgs),
        'vol badges': trans[(trans['month']==month)&(trans['org'].isin(these_orgs))]['amount'].sum(),
        'vol no badges': trans[(trans['month']==month)&(~trans['org'].isin(these_orgs))]['amount'].sum()
    })

In [ ]:
df_p2p.merge(pd.DataFrame(monthly_vol), on='month')

In [ ]:
participants = trans[(trans['transdonationentitytype']==10)&(trans['transdonationentity']!=0)].groupby(['month', 'transdonationentity'])['amount'].sum().reset_index()
participants.groupby('month')['amount'].agg(['sum', 'mean', 'median']).reset_index()

# 5. auctions

- Total \# of auction events
- Total amount raised (reg. + donations)
- Total registrations
- Total donations (online & offline)
- Total number of bids
- Avg. \# of bids

In [ ]:
# processing info
q = '''select
        t.month,
        count(distinct(t.form)) as active_auctions,
        sum(t.amount) as auction_vol,
        count(t.id) as auction_count,
        sum(t.registrations_count) as auction_registrations_count,
        sum(t.registrations_amt) as auction_registrations_vol,
        sum(t.donations_count) as auction_donations_count,
        sum(t.donations_amt) as auction_donations_vol,
        sum(t.purchases_count) as auction_purchases_count,
        sum(t.purchases_amt) as auction_purchases_vol,
        sum(t.amount) - (sum(t.donations_amt) + sum(t.purchases_amt)) as auction_reg_vol_alt
    from transactions as t
        left join form as f on t.form=f.id
    where
        t.status='A' and
        t.recurring=0 and
        f.type=5 and
        t.year>=2024
    group by t.month'''
df_auction = redshift_query_read(q, schema='production')

In [ ]:
df_auction.sort_values('month')

In [ ]:
# source: givi vs web vs vt
q = '''select
            t.month,
            t.source,
            count(t.id) as transactions,
            sum(t.amount) as volume
        from transactions as t
            left join form as f on t.form=f.id
        where
            t.status='A' and
            t.recurring=0 and
            f.type=5 and
            t.year>=2024
        group by t.month, t.source'''
auction_source = redshift_query_read(q, schema='production')

In [ ]:
source_months = auction_source.pivot(index='month', columns='source', values=['transactions', 'volume']).reset_index()
source_months.columns = ['month', 'trans don_form', 'trans_givi',
                       'trans_mobilevt', 'trans_vt', 'vol_don_form',
                       'vol_givi', 'vol_mobilevt', 'vol_vt']
source_months.fillna(0)